# MA486 Project 4: Predicting RASP Graduation
### Andrew Efaw, Vedin Fowler, Huram-Abi Nzia Yotchoum, Radford Swent

Cover page and works cited attached.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ANN import weight_bias_init_Nlayers, forward_propagation, N_layer_FW_Prop, N_layer_BWD_Prop, N_layer_weights_update, train_NN, model_predict, train_NN_final_model
from datetime import timedelta 
import pickle

First, we start by formatting the data. These functions will allow us to format the data before our neural network processes it, and will allow us to transform LTC Clark's data into a useable format. Of note, we filled missing data in 2 ways:
    
    1) If the data was for a physical event, we filled with group mean of gender and graduation
    
    2) Otherwise, we filled with the graduation group mean
    
    
We also standardized our data so that the neural network has an easier time understanding it. We encoded sex numerically, but otherwise chose to drop categorical columns.

In [2]:
RASP_data = pd.read_csv("train_data.csv")

def seconder(x):
    """
    A function that accepts a column of data that is measured in mins:seconds:milliseconds (eg 2:34:56)
        Inputs: x: a list or column of a pandas dataframe with time measurements
        Outputs: returns a list of all time entries in seconds
    """
    time  =  str(x).split(":")
    if len(time) == 3:
            mins, secs, mil = time
            td = timedelta(minutes = float(mins), seconds = float(secs), milliseconds = float(mil))
            return td.total_seconds()
    elif len(time) == 2:
            mins, secs = time
            td = timedelta(minutes = float(mins), seconds = float(secs))
            return td.total_seconds()
    else:
        return None
    
def format_data(df):
    """
    Takes dataframe in the format we are given and formats it to work with our functions
    """
    if not isinstance(df["Run"][0], np.float64):
        df["Run"] = df["Run"].apply(seconder)
    columns = list(df.columns)
    for col in columns:
        #numerically encode sex
        if col == "SEX":
            df.loc[df['SEX'] == "F", 'SEX_ENC'] =  1
            df.loc[df['SEX'] == "M", 'SEX_ENC'] = 0
        #Here we delete the non numerical columns
        if df[f'{col}'].dtypes == np.dtype('O'):
            del df[f'{col}']
    #Assume all cases where we do not know the sex that the candidate is male
    df['SEX_ENC'] = df['SEX_ENC'].fillna(0)
    #fill NA values for physical events with the group mean for sex and whether they passed/not
    df[['SEX_ENC', 'Graduate', 'Pushup','Situp','Run','Chinup']] = df[['SEX_ENC', 'Graduate', 'Pushup', 'Situp', 'Run', 'Chinup']].fillna(df[['SEX_ENC', 'Graduate', 'Pushup', 'Situp', 'Run', 'Chinup']].groupby(['SEX_ENC', 'Graduate']).transform('mean'))
    #for everything else, just fill with general mean
    df = df.fillna(df.groupby('Graduate').transform('mean'))
    #These are the columns that will be used in our model, in proper order
    cols = ['SEX_ENC', 'Married', 'Airborne', 'Age', 'Education',  'GT Score', 'Pushup', 'Situp', 'Run', 'Chinup', 'Sick Call', 'Information', 'Comprehension', 'Arithmetic', 'Similarities', 'Vocabulary', 'DS', 'PC', 'Spatial', 'PA', 'OA', 'V', 'P', 'FS', 'VRINr', 'TRINr', 'Fr', 'Fpr', 'Fs', 'FBSr', 'RBS', 'Lr', 'Kr', 'EID', 'THD', 'BXD', 'RCd', 'RC1', 'RC2', 'RC3', 'RC4', 'RC6', 'RC7', 'RC8', 'RC9', 'MLS', 'HPC', 'NUC', 'GIC', 'SUI', 'HLP', 'SFD', 'NFC', 'COG', 'STW', 'AXY', 'ANP', 'BRF', 'MSF', 'JCP', 'SUB', 'AGG', 'ACT', 'FML', 'IPP', 'SAV', 'SHY', 'DSF', 'AES', 'MEC', 'AGGRr', 'PSYCr', 'DISCr', 'NEGEr', 'INTRr', 'CS', 'PCT T', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'MTS1', 'MTS2', 'MTS3', 'MTS4', 'MTS5', 'MTS6', 'MTS7', 'MTS8', 'MTS9', 'MTS10', 'MTS11', 'MTS12', 'MTS13', 'MTS14', 'MTS15', 'FTS1', 'FTS2', 'FTS3', 'FTS4', 'FTS5', 'FTS6', 'FTS7', 'FTS8', 'FTS9', 'FTS10', 'FTS11', 'FTS12', 'FTS13', 'FTS14', 'FTS15', 'TSC1', 'TSC2', 'TSC3', 'TSC4', 'TSC5', 'TSC6', 'TSC7', 'TSC8', 'TSC9', 'TSC10','TSC11', 'TSC12', 'TSC13', 'TSC14', 'TSC15', 'MPC1', 'MPC2', 'MPC3', 'MPC4','MPC5', 'MPC6', 'MPC7', 'MPC8', 'MPC9', 'MPC10', 'MPC11', 'MPC12', 'MPC13', 'MPC14', 'MPC15', 'FPC1', 'FPC2', 'FPC3', 'FPC4', 'FPC5', 'FPC6', 'FPC7', 'FPC8', 'FPC9', 'FPC10', 'FPC11','FPC12', 'FPC13', 'FPC14', 'FPC15', 'PCT1', 'PCT2', 'PCT3', 'PCT4', 'PCT5', 'PCT6', 'PCT7', 'PCT8', 'PCT9', 'PCT10', 'PCT11', 'PCT12', 'PCT13', 'PCT14', 'PCT15', 'RCL1', 'RCL2', 'RCL3', 'RCL4', 'RCL5', 'MCL1', 'MCL2', 'MCL3', 'MCL4', 'MCL5', 'FCL1', 'FCL2', 'FCL3', 'FCL4', 'FCL5', 'BCL1', 'BCL2', 'BCL3', 'BCL4', 'BCL5', 'MCP1', 'MCP2', 'MCP3', 'MCP4', 'MCP5', 'FCP1', 'FCP2', 'FCP3', 'FCP4', 'FCP5', 'BCP1', 'BCP2', 'BCP3', 'BCP4', 'BCP5', 'RCI', 'INF', 'MIS', 'Graduate']
    return df[cols]

def standardize_data(df):
    '''
    Takes formatted data from format_data() and standardizes columns
    Inputs: df - RASP_data object after formatting
    Outputs: all columns standardized except graduate
    '''
    #remove the response variable from the data set
    grad = df["Graduate"].copy(deep = True)
    del df["Graduate"]
    #standardize the dataframe
    df = (df - df.mean(axis = 0))/df.std(axis = 0)
    #add the response variable at the end
    df["Graduate"] = grad
    return df

RASP_data = standardize_data(format_data(RASP_data))

#### Bootstrapping more datapoints

We decided to bootstrap data to get more observations. We did this by treating the graduated observations as one group and non-graduates as another group. Then, we simulated graduates and non-graduated by filling the columns in with a random choice (for categorical columns) or a normal distribution choice (for numerical data) based on the group distributions. This allows us to get a more representative and larger group of observations for RASP candidates.

In [3]:
def get_bootstrap(runFunction = False):
    """
    Allows user to run bootstrap function to bootstrap 20000 observations
    off of given train data. If user does not want to wait 20 minutes for
    this, the default is that it just reads a csv file where a bootstrap
    has already been recorded
    """
    from RASP_bootstrapper import bootstrap
    if runFunction: return bootstrap(RASP_data, {'Graduate': 1})
    return pd.read_csv("bootstrapped_RASP_data.csv")

bootstrapped_data = standardize_data(format_data(get_bootstrap()))

#### Before continuing, understand that we will break our data down to 80% train and 20% test. 

###### We chose to use $k$ Fold Cross Validation in this project on the 80% train data in order to avoid overfitting to just this dataframe. K Fold Cross Validation is a method that subsets the data into $k$, or in this case, 5 different "folds." We will then use 4 of these folds to train the model and 1 of these "folds" to test. We will do this repeatedly until each combination of folds in the train data has been used, saving the weights of neural network each time. We will take the mean of the weights in order to see how good our model is.

###### Once we have our mean weight, we will finally test on the withheld 20% of data and observe the accuracy of the model with these mean weights. We expect this 20% to be representative of LTC Clark's data and we will not have used it in training our model.

In [4]:
#This cell sends the data through our neural network and outputs the accuracy, weights, errors, and the data that was not used to train the neural network
'''
acc, ws, cost, untouched_data = train_NN(bootstrapped_data, 1000, 0.8, [300, 30])
'''

'\nacc, ws, cost, untouched_data = train_NN(bootstrapped_data, 1000, 0.8, [300, 30])\n'

Here, we average our weights, cost, and accuracy over the 5 runs. Then, we saved our weights using pickle so we would not have to run the 90 minute training function. 

In [5]:
### Averaging
# w, b  = ws[0]
# w1,b1 = ws[1]
# w2,b2 = ws[2]
# w3,b3 = ws[3]
# w4,b4 = ws[4]

# layer1_ws = (w[0] + w1[0]+ w2[0] + w3[0] + w4[0])/5
# layer2_ws = (w[1] + w1[1]+ w2[1] + w3[1] + w4[1])/5
# layer3_ws = (w[2] + w1[2]+ w2[2] + w3[2] + w4[2])/5

# layer1_b = (b[0] + b1[0]+ b2[0] + b3[0] + b4[0])/5
# layer2_b = (b[1] + b1[1]+ b2[1] + b3[1] + b4[1])/5
# layer3_b = (b[2] + b1[2]+ b2[2] + b3[2] + b4[2])/5

# average_Ws = [layer1_ws,layer2_ws,layer3_ws]
# average_b = [layer1_b,layer2_b,layer3_b]

### used to save kfold results
# with open("kfoldWs.pkl", "wb") as f:
#     pickle.dump(average_Ws, f)

# with open("kfoldBs.pkl", "wb") as f:
#     pickle.dump(average_b, f)
    
# with open("kfoldcost.pkl", "wb") as f:
#     pickle.dump(cost, f)
    
# with open("kfoldacc.pkl", "wb") as f:
#     pickle.dump(acc, f)
# with open("untouchedata.pkl", "wb") as f:
#     pickle.dump(untouched_data, f)


In [6]:
#importing our variables from pickle files
with open('kfoldcost.pkl', 'rb') as f:
    cost = pickle.load(f)
    
with open('kfoldacc.pkl', 'rb') as f:
    acc = pickle.load(f)

with open('kfoldBs.pkl', 'rb') as f:
    avg_Bs = pickle.load(f)

with open('kfoldWs.pkl', 'rb') as f:
    avg_Ws = pickle.load(f)
with open('untouchedata.pkl', 'rb') as f:
    ut_data = pickle.load(f)

In [7]:
#Accuracy check against untesested data from the Kfold cross validation 
ut_data = np.array(ut_data)
Xtest = ut_data[:,0:-1].T
Ytest = ut_data[:,-1]
y_out, y_pred = model_predict(Xtest, avg_Ws, avg_Bs, 0.382, 4)
acc = np.mean(y_pred==Ytest)
print(acc)

0.9072434858496446


In [8]:
#Training final model on full bootsrapped dat 80-20 train/test split. 

# selects = np.random.rand(len(bootstrapped_data)) < 0.8

# X = np.array(bootstrapped_data[selects].iloc[:,0:-1])
# Y = np.array(bootstrapped_data[selects].iloc[:,-1])
# Xtest = np.array(bootstrapped_data[~selects].iloc[:,0:-1])
# Ytest = np.array(bootstrapped_data[~selects].iloc[:,-1])


# fWs, fbs, n, fpred_err, fcosts = train_NN_final_model(X.T, Y, 10000, 0.8, Xtest.T, Ytest, [300,30])

### used to save final results
# with open("FinalWs.pkl", "wb") as f:
#     pickle.dump(fWs, f)

# with open("FinalBs.pkl", "wb") as f:
#     pickle.dump(fBs, f)
    
# with open("finalcost.pkl", "wb") as f:
#     pickle.dump(fcosts, f)
    
# with open("finalerr.pkl", "wb") as f:
#     pickle.dump(fpred_err, f) 

#### Graphing the error

We graphed the error in order to ensure that over the course of fitting weights to our ANN, we see strictly decreasing error, indicating a lack of overfitting.

In [9]:
#This code averages the error across all 5 folds and graphs it to look for potential overfitting
def graph_avg_vals(errors):
    """
    This function accepts the error vectors from train_NN and averages the errors across the multiple folds on the data. It then graphs the errors to see if there is any overfitting on the data.
    Inputs: -errors: The error matrix (in the form of a list of lists) that contains the SSE for all the folds
    Outputs: A graph of the average error across all the folds.
    """
    y=[]
    for i in range(len(errors[0])):
        y.append([])
    for idx,ycol in enumerate(errors):
        for i,yval in enumerate(ycol):
            y[i].append(yval)
    for i,vals in enumerate(y):
        y[i]=sum(vals)/len(vals)
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.set_ylabel("SSE")
    ax.set_xlabel("Iterations Through the ANN")
    ax.set_title("Graph of SSE Over Iterations Through the ANN With Kfold Cross validation")
    plt.plot([i for i in range(len(errors[0]))], y)
    plt.show()

In [10]:
graph_avg_vals(cost)

In [11]:
with open("cost.pkl", "rb") as f:
    costss = pickle.load(f)
    
with open("pred_err.pkl", "rb") as f:
    pred_err = pickle.load(f)

In [12]:
pd.Series(costss).plot(legend=True, label="Sum Square Error of Training Set")
pd.Series(pred_err).plot(legend=True, label="Test Prediction Error")
plt.title("SSE of Training Set and Test Prediction Error ")
plt.show()

# User Interface

###### This is where you come in, sir. Below, we will ask you to import your data, we will format it, and our algorithm will run on it. This will consist of taking our derived weights and predicting outcomes of your data based on our weights. Then, we will tell you how accurately we were able to predict graduation rates of the people in your data based on  the model produced by our neural network.

In [17]:
def user_interface():
    """
    This function assumes the data is in the local working directory of this file (in this case the data is in Project 4 folder) and is a .csv.
    Inputs: None.
    Outputs: Prints accuracy, returns predictions and probabilities vector.
    """
    #This imports the data
    dat_name = input('Please input the string name of your data (in the form of "csv_name.csv"):')
    dat = pd.read_csv(dat_name)
    #format and standardize the data so it works with our neural network
    dat = format_data(dat)
    dat = standardize_data(dat)
    dat_np = np.array(dat.iloc[:,0:-1])
    #Import weights (this is done because it will take too long to train the model [~90 mins]),
    #so we already ran it and saved the weights to a pickle file. This uses our train_NN_full_modelX
    with open("FullWs.pkl", "rb") as f:
        Ws = pickle.load(f)
    with open("FullBs.pkl", "rb") as f:
        Bs = pickle.load(f)
    #get predictions based on the weights. Tries two different threshhold values
    y_prob1, y_pred1 = model_predict(dat_np.T, Ws, Bs, 0.5, 4)
    y_prob2, y_pred2 = model_predict(dat_np.T, Ws, Bs, 0.382, 4)
    y = np.array(dat['Graduate'])
    #accuracy check
    acc1 = np.mean(y_pred1==y)
    acc2 = np.mean(y_pred2==y)
    '''
    if acc1 > acc2:
        print(f'{acc1*100:.2f}%')
        return y_pred1, y_prob1
    '''
    print(f'{acc2*100:.2f}%', f'{acc1*100:.2f}%')
    
    return acc2,acc1
    
results = user_interface()

Please input the string name of your data (in the form of "csv_name.csv"): 

(0.7469879518072289, 0.7521514629948365)